Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia¶

In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled List of Falcon 9 and Falcon Heavy launches

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches

Objectives¶
Web scrap Falcon 9 launch records with BeautifulSoup:

Extract a Falcon 9 launch records HTML table from Wikipedia
Parse the table and convert it into a Pandas data frame

In [2]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the List of Falcon 9 and Falcon Heavy launches Wikipage updated on 9th June 2021

In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

TASK 1: Request the Falcon9 Launch Wiki page from its URL

In [9]:
import requests
from bs4 import BeautifulSoup

# URL della pagina
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Headers per simulare un browser reale
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

# 1. Richiedi la pagina con requests.get()
response = requests.get(static_url, headers=headers)

# 2. Crea un oggetto BeautifulSoup dal contenuto HTML della risposta
soup = BeautifulSoup(response.text, "html.parser")

# 3. Stampa il titolo della pagina per verificare
print("Page title:", soup.title.string)


Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


In [10]:
# 1. Trova tutte le tabelle nella pagina con BeautifulSoup
html_tables = soup.find_all("table")

print("Numero di tabelle trovate:", len(html_tables))

# 2. Seleziona la terza tabella (indice 2 perché Python parte da 0)
first_launch_table = html_tables[2]

# 3. Stampa la tabella per controllarne il contenuto
print(first_launch_table)


Numero di tabelle trovate: 25
<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage

In [12]:
# Lista per salvare i nomi delle colonne
column_names = []

# Trova tutti gli elementi <th> nella tabella
table_headers = first_launch_table.find_all("th")

# Itera su ciascun <th> ed estrai il nome colonna
for th in table_headers:
    name = extract_column_from_header(th)   # funzione fornita nel lab
    if name is not None and len(name) > 0:
        column_names.append(name)

# Controlla i nomi delle colonne estratte
print("Column names:", column_names)


Column names: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


TASK 3: Create a data frame by parsing the launch HTML tables

We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe

In [13]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the launch_dict with launch records extracted from table rows.

Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links B0004.1[8], missing values N/A [e], inconsistent formatting, etc.

To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the launch_dict. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:

In [21]:
launch_dict = {
    "Flight No.": [],
    "Date and Time (UTC)": [],
    "Launch Site": [],
    "Payload": [],
    "Payload Mass": [],
    "Orbit": [],
    "Customer": [],
    "Launch Outcome": [],
    "Rocket": []
}

# Trova tutte le righe della tabella (tr)
table_rows = first_launch_table.find_all("tr")

for row in table_rows:
    # Trova tutte le celle della riga (td)
    cells = row.find_all("td")

    # Se la riga ha il numero giusto di celle (9 in questo caso)
    if len(cells) == 9:
        # Estrarre e pulire i valori cella per cella
        flight_no = cells[0].get_text(strip=True)
        date_time = cells[1].get_text(strip=True)
        launch_site = cells[2].get_text(strip=True)
        payload = cells[3].get_text(strip=True)
        payload_mass = cells[4].get_text(strip=True).replace("\xa0", " ")
        orbit = cells[5].get_text(strip=True)
        customer = cells[6].get_text(strip=True)
        launch_outcome = cells[7].get_text(strip=True)
        rocket = cells[8].get_text(strip=True)

        # Riempire il dizionario
        launch_dict["Flight No."].append(flight_no)
        launch_dict["Date and Time (UTC)"].append(date_time)
        launch_dict["Launch Site"].append(launch_site)
        launch_dict["Payload"].append(payload)
        launch_dict["Payload Mass"].append(payload_mass)
        launch_dict["Orbit"].append(orbit)
        launch_dict["Customer"].append(customer)
        launch_dict["Launch Outcome"].append(launch_outcome)
        launch_dict["Rocket"].append(rocket)

print("Numero di record estratti:", len(launch_dict["Flight No."]))


Numero di record estratti: 7


In [22]:
# Converti il dizionario in DataFrame
df = pd.DataFrame(launch_dict)

# Mostra le prime 5 righe per controllare
print(df.head())

                  Flight No.   Date and Time (UTC)   Launch Site  \
0          4 June 2010,18:45  F9 v1.0[7]B0003.1[8]  CCAFS,SLC-40   
1  8 December 2010,15:43[13]  F9 v1.0[7]B0004.1[8]  CCAFS,SLC-40   
2      22 May 2012,07:44[17]  F9 v1.0[7]B0005.1[8]  CCAFS,SLC-40   
3   8 October 2012,00:35[21]  F9 v1.0[7]B0006.1[8]  CCAFS,SLC-40   
4         1 March 2013,15:10  F9 v1.0[7]B0007.1[8]  CCAFS,SLC-40   

                                  Payload           Payload Mass     Orbit  \
0    Dragon Spacecraft Qualification Unit                              LEO   
1       Dragondemo flight C1(Dragon C101)                         LEO(ISS)   
2  Dragondemo flight C2+[18](Dragon C102)  525 kg (1,157 lb)[19]  LEO(ISS)   
3           SpaceX CRS-1[22](Dragon C103)   4,700 kg (10,400 lb)  LEO(ISS)   
4           SpaceX CRS-2[22](Dragon C104)   4,877 kg (10,752 lb)  LEO(ISS)   

        Customer Launch Outcome                     Rocket  
0         SpaceX        Success  Failure[9][10](parachute)  


In [15]:
""""extracted_row = 0
#Extract each table
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element
        row=rows.find_all('td')
        #if it is number save cells in a dictonary
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])

            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)

            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)

            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)

            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)

            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)

            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)

            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)

            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)

            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)

            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)"""


F9 v1.07B0003.18
F9 v1.07B0004.18
F9 v1.07B0005.18
F9 v1.07B0006.18
F9 v1.07B0007.18
F9 v1.17B10038
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 FT[
F9 v1.1[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT♺[
F9 FT[
F9 FT[
F9 FT[
F9 FTB1029.2195
F9 FT[
F9 FT[
F9 B4[
F9 FT[
F9 B4[
F9 B4[
F9 FTB1031.2220
F9 B4[
F9 FTB1035.2227
F9 FTB1036.2227
F9 B4[
F9 FTB1032.2245
F9 FTB1038.2268
F9 B4[
F9 B4B1041.2268
F9 B4B1039.2292
F9 B4[
F9 B5311B1046.1268
F9 B4B1043.2322
F9 B4B1040.2268
F9 B4B1045.2336
F9 B5
F9 B5349B1048[
F9 B5B1046.2354
F9 B5[
F9 B5B1048.2364
F9 B5B1047.2268
F9 B5B1046.3268
F9 B5[
F9 B5[
F9 B5B1049.2397
F9 B5B1048.3399
F9 B5[]413
F9 B5[
F9 B5B1049.3434
F9 B5B1051.2420
F9 B5B1056.2465
F9 B5B1047.3472
F9 B5
F9 B5[
F9 B5B1056.3482
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5[
F9 B5
F9 B5
F9 B5
F9 B5B1058.2544
F9 B5
F9 B5B1049.6544
F9 B5
F9 B5B1060.2563
F9 B5B1058.3565
F9 B5B1051.6568


AttributeError: 'NoneType' object has no attribute 'string'

After you have fill in the parsed launch record values into launch_dict, you can create a dataframe from it.

In [23]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [24]:
#We can now export it to a CSV for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

#Following labs will be using a provided dataset to make each lab independent.

df.to_csv('spacex_web_scraped.csv', index=False)

In [25]:
from google.colab import files

# Scarica il file CSV sul tuo computer
files.download("spacex_web_scraped.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>